In [1]:
import dlib

import cv2 as cv
import cv2
import os
import re
import json
from pylab import *
from PIL import Image, ImageChops, ImageEnhance
import fn
import glob
import pandas as pd
import face_recognition

In [2]:
file = glob.glob(f'/projects/jbaldo/Deepfake/dfdc_train_part_*9/*.mp4')

In [3]:
path = glob.glob('/projects/jbaldo/Deepfake/dfdc_train_part_*/metadata.json')
dl=[]
for i in path:
    dl.append(pd.read_json(i).T)
df=pd.concat(dl)
df.head()

,label,split,original
cveqgferbf.mp4,REAL,train,NaN
dskhcyjnqw.mp4,FAKE,train,mzmnjmwunq.mp4
bcdmnkzrqa.mp4,FAKE,train,mzmnjmwunq.mp4
jzrdfehytl.mp4,FAKE,train,tyaakhhemf.mp4
rkbautpdzv.mp4,FAKE,train,emrsqvjawa.mp4


In [4]:
Fake=df[df['label']=='FAKE']
Real=df[df['label']=='REAL']
Fake.drop_duplicates(subset='original', inplace=True)
df_clean=pd.concat([Fake,Real],axis=0)
len(df_clean)

C:\Users\qazya\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


1728

In [11]:
def cap_img(file):
    video_file = file
    cap = cv2.VideoCapture(video_file)
    frames = []
    c = 0
    timeF = 60
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret==True:
            if c%timeF == 0: 
                    frames.append(frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
            c+=1
        else:
            break
    cap.release()
    return frames

In [14]:
for f in file:
    name=f.split('/')[-1]
    if name in df_clean.index:
        label = df.loc[name]["label"]
        frame=cap_img(f)
        for n,f in enumerate(frame):
            image = cv.cvtColor(f, cv.COLOR_BGR2RGB)
            face_locations = face_recognition.face_locations(image)
            for num,face_location in enumerate(face_locations):
                top, right, bottom, left = face_location
                face_image = image[top:bottom, left:right]
                if label == 'REAL':    
                    cv2.imwrite("/projects/jbaldo/Deepfake/Images11/Real/"+name.replace(".mp4","")+str(n)+".png",cv2.resize(face_image, (128, 128)))
                elif label =='FAKE':
                    cv2.imwrite("/projects/jbaldo/Deepfake/Images11/Fake/"+name.replace(".mp4","")+str(n)+".png",cv2.resize(face_image, (128, 128)))